The notebook load the dataset from the Google drive and does inference

In [1]:
# Install all the required packages (borrowed from openWakeWord's automatic training notebook)
running_on_colab = False
finetune = False # start training from larger weights or with smaller finetuning weights

if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    from google.colab import drive
    drive.mount('/content/drive')
    running_on_colab = True
    # this variable determine if a traing from scratch or continued traing is desired
    restore_model = False


In [2]:
if running_on_colab:
  # restoring previous trained model for finetuning from previously generated features
  trainging_features_filename = "training_features_20240421_155624.tar"
  trained_model_filename = "trained_models_20240418_181306.tar"
  !cp /content/drive/MyDrive/ColabNotebooks/VoiceAssistant/microWakeWord/{trainging_features_filename} .
  !tar -xvf ./{trainging_features_filename}
  if restore_model:
    !cp /content/drive/MyDrive/ColabNotebooks/VoiceAssistant/microWakeWord/{trained_model_filename}  .
    !tar -xvf ./{trained_model_filename}

In [3]:
# Install necessary packages
#%pip install -q tensorflow
%pip install -q mmap_ninja
%pip install -q pyyaml
%pip install -q datasets
%pip install -q tensorflow==2.15.0
%pip install -q keras==2.15.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
if not os.path.isdir("./microWakeWord"):
    !git clone https://github.com/kahrendt/microWakeWord
    %pip install -e ./microWakeWord

print("Packages installed successfully!")
print("Please restart the kernel to use the installed packages.")

Obtaining file:///home/lior/Custom-MicroWakeWord-Generator/microWakeWord
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for microwakeword (pyproject.toml) ... done
  Created wheel for microwakeword: filename=microwakeword-0.0.1-0.editable-py3-none-any.whl size=12181 sha256=bbde1758d22e0b9684f859a7f5e29a561aa28952cd0e995a2be9942b58fcbfa4
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-z62lzy_9/wheels/c6/4e/97/e37960bb6fb6bbc1a8210b78076f899bc6709a86b83d65bbfd
Successfully built microwakeword
  Attempting uninstall: microwakeword
    Found existing installation: microwakeword 0.0.1
    Uninstalling microwakeword-0.0.1:
      Successfully uninstalled microwakeword-0.0.1
Note: you may need to restart the kernel to use updated packages.
Packages installed successfu

In [8]:
# Save a yaml config that controls the training process
import yaml
import os

config = {}

config['train_dir'] = 'trained_models/alexha'

# Each feature_dir should have at least one of the following folders with this structure:
#  training/
#    ragged_mmap_folders_ending_in_mmap
#  testing/
#    ragged_mmap_folders_ending_in_mmap
#  testing_ambient/
#    ragged_mmap_folders_ending_in_mmap
#  validation/
#    ragged_mmap_folders_ending_in_mmap
#  validation_ambient/
#    ragged_mmap_folders_ending_in_mmap
#
#  sampling_weight: Weight for choosing a spectrogram from this set in the batch
#  penalty_weight: Penalizing weight for incorrect predictions from this set
#  truth: Boolean whether this set has positive samples or negative samples
#  truncation_strategy = If spectrograms in the set are longer than necessary for training, how are they truncated
#       - random: choose a random portion of the entire spectrogram - useful for long negative samples
#       - truncate_start: remove the start of the spectrogram
#       - truncate_end: remove the end of the spectrogram
#       - split: Split the longer spectrogram into separate spectrograms offset by 100 ms. Only for ambient sets

config['features'] = [
        {
            'features_dir': './training_features/positive_tts_samples',
            'sampling_weight': 1,
            'penalty_weight': 1,
            'truth': True,
            'truncation_strategy': 'truncate_start',
            'type': "mmap",
        },
        {
            'features_dir': './training_features/negative_tts_samples',
            'sampling_weight': 1,
            'penalty_weight': 1,
            'truth': False,
            'truncation_strategy': 'truncate_start',
            'type': "mmap",
        },
        {
            'features_dir': './training_features/positive_voice_samples',
            'sampling_weight': 1,
            'penalty_weight': 3,
            'truth': True,
            'truncation_strategy': 'truncate_start',
            'type': "mmap",
        },
        {
            'features_dir': './training_features/positive_cloned_samples',
            'sampling_weight': 1,
            'penalty_weight': 1,
            'truth': True,
            'truncation_strategy': 'truncate_start',
            'type': "mmap",
        },
        {
            'features_dir': './training_features/positive_pronounciation_samples',
            'sampling_weight': 1,
            'penalty_weight': 1,
            'truth': True,
            'truncation_strategy': 'truncate_start',
            'type': "mmap",
        },        
        # {
        #     'features_dir': '/Volumes/MachineLearning/training_data/negative_datasets/english_speech_background',
        #     'sampling_weight': 2,
        #     'penalty_weight': 1,
        #     'truth': False,
        #     'truncation_strategy': 'random',
        #     'type': "mmap",
        # },
        # {
        #     'features_dir': '/Volumes/MachineLearning/training_data/negative_datasets/non_english_speech_background',
        #     'sampling_weight': 1,
        #     'penalty_weight': 1,
        #     'truth': False,
        #     'truncation_strategy': 'random',
        #     'type': "mmap",
        # },
        # {
        #     'features_dir': '/Volumes/MachineLearning/training_data/negative_datasets/no_speech_background',
        #     'sampling_weight': 1,
        #     'penalty_weight': 1,
        #     'truth': False,
        #     'truncation_strategy': 'random',
        #     'type': "mmap",
        # },

        {
            'features_dir': './training_features/wham_noise',
            'sampling_weight': 1,
            'penalty_weight': 3,
            'truth': False,
            'truncation_strategy': 'split',
            'type': "mmap",
        },
        # {
        #     'features_dir': './training_features/but_reverdb_silence',
        #     'sampling_weight': 1,
        #     'penalty_weight': 1,
        #     'truth': False,
        #     'truncation_strategy': 'split',
        #     'type': "mmap",
        # },
        {
            'features_dir': './training_features/dipco',
            'sampling_weight': 2,
            'penalty_weight': 3,
            'truth': False,
            'truncation_strategy': 'random',
            'type': "mmap",
        },
    ]

# Number of training steps in each iteration - various other settings are configured as lists that corresponds to different steps
if finetune==True:
  config['training_steps'] = [0,0,0,10000]  # [30000, 30000,20000,20000]
else:
  config['training_steps'] = [1000,1000,2000,3000]  # [30000, 30000,20000,20000]


# Penalizing weight for incorrect class predictions - lists that correspond to training steps
config["positive_class_weight"] = [1]
config["negative_class_weight"] = [1]

config['learning_rates'] = [0.001, 0.0005,0.0002,0.0001] # Learning rates for Adam optimizer - list that corresponds to training steps
config['batch_size'] = 1024

config['mix_up_augmentation_prob'] =  [0]       # Probability of applying MixUp augmentation - list that corresponds to training steps
config['freq_mix_augmentation_prob'] = [0]      # Probability of applying FreqMix augmentation - list that corresponds to training steps
config['time_mask_max_size'] = [5]              # SpecAugment - list that corresponds to training steps
config['time_mask_count'] = [2]                 # SpecAugment - list that corresponds to training steps
config['freq_mask_max_size'] = [5]              # SpecAugment - list that corresponds to training steps
config['freq_mask_count'] = [2]                 # SpecAugment - list that corresponds to training steps
config['eval_step_interval'] = 200 #500              # Test the validation sets after every this many steps

config['clip_duration_ms'] = 1490   # Maximum length of wake word that the streaming model will accept
config['window_stride_ms'] = 20     # Fixed setting for default feature generator
config['window_size_ms'] = 30       # Fixed setting for default feature generator
config['sample_rate'] = 16000       # Fixed setting for default feature generator

# The best model weights are chosen first by minimizing the specified minimization metric below the specified target_minimization
# Once the target has been met, it chooses the maximum of the maximization metric. Set 'minimization_metric' to None to only maximize
# Available metrics:
#   - "loss" - cross entropy error on validation set
#   - "accuracy" - accuracy of validation set
#   - "recall" - recall of validation set
#   - "precision" - precision of validation set
#   - "false_positive_rate" - false positive rate of validation set
#   - "false_negative_rate" - false negative rate of validation set
#   - "ambient_false_positives" - count of false positives from the split validation_ambient set
#   - "ambient_false_positives_per_hour" - estimated number of false positives per hour on the split validation_ambient set
config['minimization_metric'] = 'ambient_false_positives_per_hour'  # Set to None to disable
config['target_minimization'] = 0.5
config['maximization_metric'] = 'recall'
config['binary_classification'] = False

with open(os.path.join('training_parameters.yaml'), 'w') as file:
    documents = yaml.dump(config, file)

In [9]:

!python3 -m microwakeword.model_train_eval \
--training_config='training_parameters.yaml' \
--train 1 \
--restore_checkpoint 1 \
--test_tf_nonstreaming 0 \
--test_tflite_nonstreaming 0 \
--test_tflite_streaming 0 \
--test_tflite_streaming_quantized 1 \
--use_weights "last_weights" \
inception \
--cnn1_filters '32' \
--cnn1_kernel_sizes '5' \
--cnn1_subspectral_groups '1' \
--cnn2_filters1 '24,24,24' \
--cnn2_filters2 '32,64,96' \
--cnn2_kernel_sizes '3,5,5' \
--cnn2_subspectral_groups '1,1,1' \
--cnn2_dilation '1,1,1' \
--dropout 0.8


2024-04-29 18:57:26.238728: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 18:57:26.238783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 18:57:26.239984: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 18:57:26.247189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:absl:Loading and analyzing dat

Backup model for later continued training

In [ ]:
!tar -cvf ./trained_models_$(date +%Y%m%d_%H%M%S).tar ./trained_models
!cp -r ./trained_models*.tar /content/drive/MyDrive/ColabNotebooks/VoiceAssistant/microWakeWord/